In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, cross_val_score

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import (accuracy_score, 
                            confusion_matrix, 
                            classification_report,
                            f1_score,
                            plot_confusion_matrix,
                            precision_recall_curve,
                            precision_score,
                            recall_score)

In [93]:
pwd

'/Users/emilynaftalin/Data_Science/General Assembly/dsi/projects/Mass-Protests/users'

In [4]:
mass = pd.read_pickle('../data/03_dummy_df.pickle')

In [22]:
mass.head(2)

id country  ccode         region  protestnumber  protesterviolence  \
0  201990001  Canada     20  North America              1                0.0   
1  201990002  Canada     20  North America              2                0.0   

           location protesteridentity  \
0          national       unspecified   
1  Montreal, Quebec       unspecified   

                                             sources  \
0  1. great canadian train journeys into history;...   
1  1. autonomy s cry revived in quebec the new yo...   

                                               notes  ...  \
0  canada s railway passenger system was finally ...  ...   
1  protestors were only identified as young peopl...  ...   

   protester_id_type_political_group  protester_id_type_prisoners  \
0                                  0                            0   
1                                  0                            0   

   protester_id_type_protestors_generic  protester_id_type_religious_group  \
0                                     0                                  0   
1                                     0                                  0   

   protester_id_type_soldiers_veterans  protester_id_type_students_youth  \
0                                    0                                 0   
1                                    0                                 1   

   protester_id_type_victims_families  protester_id_type_women  \
0                                   0                        0   
1                                   0                        0   

   protester_id_type_workers_unions  protester_id_type_nan  
0                                 1                      0  
1                                 0                      0  

[2 rows x 227 columns]

In [64]:
sorted(list(mass.columns))

['accomodation',
 'arrests',
 'beatings',
 'ccode',
 'country',
 'country_Afghanistan',
 'country_Albania',
 'country_Algeria',
 'country_Angola',
 'country_Argentina',
 'country_Armenia',
 'country_Austria',
 'country_Azerbaijan',
 'country_Bahrain',
 'country_Bangladesh',
 'country_Belarus',
 'country_Belgium',
 'country_Benin',
 'country_Bolivia',
 'country_Bosnia',
 'country_Botswana',
 'country_Brazil',
 'country_Bulgaria',
 'country_Burkina Faso',
 'country_Burundi',
 'country_Cambodia',
 'country_Cameroon',
 'country_Canada',
 'country_Cape Verde',
 'country_Central African Republic',
 'country_Chad',
 'country_Chile',
 'country_China',
 'country_Colombia',
 'country_Comoros',
 'country_Congo Brazzaville',
 'country_Congo Kinshasa',
 'country_Costa Rica',
 'country_Croatia',
 'country_Cuba',
 'country_Cyprus',
 'country_Czech Republic',
 'country_Czechoslovakia',
 'country_Denmark',
 'country_Djibouti',
 'country_Dominican Republic',
 'country_Ecuador',
 'country_Egypt',
 'count

In [7]:
# dummifying protest_size and protester_id_type columns 
mass = pd.get_dummies(mass, columns=['protest_size_category', 'protester_id_type'], dummy_na=True)

Below, attempting a Logistic Regression for each state response, starting with `arrests`.

In [9]:
# X = mass.drop(columns=['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
#                               'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings'])
# y = mass['arrests']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

# ss = StandardScaler()

# X_train_sc = ss.fit_transform(X_train)
# X_test_sc = ss.transform(X_test)

# lr = LogisticRegression(solver='lbfgs', max_iter=1000)

# lr.fit(X_train_sc, y_train)

LogisticRegression(max_iter=1000)

In [10]:
# lr.score(X_train_sc, y_train)

0.8637931034482759

In [11]:
# lr.score(X_test_sc, y_test)

0.851888341543514

Now that I have created and evaluated the Logistic Regression model using the training and test set, I will scale the entire `X` set and make predictions for each row in the DataFrame. This will allow me to add two new columns: 
+ **`arrests_predicted`**: column of 0s and 1s indicating whether the protest is predicted to have arrests as one of the state responses. 
+ **`arrests_probability`**: the calculated probability that the protest will have arrests as one of the state responses

In [12]:
# ss = StandardScaler()

# X_sc = ss.fit_transform(X)

In [14]:
# arrests_predicted_all = lr.predict(X_sc)

# arrests_probability_all = lr.predict_proba(X_sc)[:,1]

In [15]:
# mass['arrests_predicted'] = arrests_predicted_all

# mass['arrests_probability'] = arrests_probability_all 

In [52]:
# checking out how it looks 
# mass[['arrests', 'arrests_predicted','arrests_probability']][22:33]

In [8]:
mass.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15225 entries, 0 to 15224
Columns: 227 entries, id to protester_id_type_nan
dtypes: datetime64[ns](2), float64(6), int32(180), int64(11), object(6), uint8(22)
memory usage: 13.8+ MB


I will create three functions: 

In [94]:
def evaluate_model(y_true, y_preds):
        
    f1 = f1_score(y_true, y_preds)
    
    precision = precision_score(y_true, y_preds)
    
    recall = recall_score(y_true, y_preds)

    return f1, precision, recall

In [34]:
# def build_confusion_matrix(y_true, y_preds, title):
    
#     tn, fp, fn, tp = confusion_matrix(y_true, y_preds).ravel()
#     disp = plot_confusion_matrix(model, 
#                                  X_test_sc, 
#                                  y_test, 
# #                                  display_labels=
#                                  cmap="Blues")
    
#     disp.ax_.set_title(title)
#     print(title)
#     print(disp.confusion_matrix)
    
#     plt.show
    
#     return

In [98]:
def response_prediction_columns(model, df, features):
    
    ss = StandardScaler()
    
    X = features
    X_sc = ss.fit_transform(X)
    
    predicted_column = model.predict(X_sc)
    probability_column = model.predict_proba(X_sc)[:,1]
    
    return predicted_column, probability_column

# ss = StandardScaler()

# X_sc = ss.fit_transform(X)

# arrests_predicted_all = lr.predict(X_sc)

# arrests_probability_all = lr.predict_proba(X_sc)[:,1]

# mass['arrests_predicted'] = arrests_predicted_all

# mass['arrests_probability'] = arrests_probability_all 

In [99]:
def state_response_predictor(df, features, target, model):
    
    X = features 
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

    ss = StandardScaler()

    X_train_sc = ss.fit_transform(X_train)
    X_test_sc = ss.transform(X_test)
    
    model.fit(X_train_sc, y_train)
    
    y_preds_train = model.predict(X_train_sc)
    y_preds_test = model.predict(X_test_sc)
    
    f1_score_train, precision_train, recall_train = evaluate_model(y_train, y_preds_train)
    f1_score_test, precision_test, recall_test = evaluate_model(y_test, y_preds_test)
    
    print(f'{target}- F1_score for {model} model, train set: {f1_score_train}')
    print(f'{target}- F1_score for {model} model, test set: {f1_score_test}')
    print(f'{target}- Precision for {model} model, train set: {precision_train}')
    print(f'{target}- Precision for {model} model, test set: {precision_test}')
    print(f'{target}- Recall for {model} model, train set: {recall_train}')
    print(f'{target}- Recall for {model} model, teset set: {recall_test}')
    
    predicted_column, probability_column = response_prediction_columns(model, df, features)
    
    print(model)

#     build_confusion_matrix(y_test, y_preds_test)

    return predicted_column, probability_column, f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test  

In [68]:
mass['arrests_predicted'], mass['arrests_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features_1, 'arrests', LogisticRegression_1)

arrests- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.2027871215761653
arrests- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.18148820326678766
arrests- Precision for LogisticRegression(max_iter=1000) model, train set: 0.5368956743002544
arrests- Precision for LogisticRegression(max_iter=1000) model, test set: 0.5434782608695652
arrests- Recall for LogisticRegression(max_iter=1000) model, train set: 0.125
arrests- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.10893246187363835
LogisticRegression(max_iter=1000)


In [66]:
# mass.drop(columns=['arrests_predicted','arrests_probability'], inplace=True)

In [ ]:
# mass['arrests_predicted'], mass['arrests_probability'] = response_prediction_columns(lr_arrests_1, mass, features_1)

_Implementing three above functions for each state response. Will be tweaking the model throughout._

In [100]:
# features_1

In [101]:
features_1 = mass.drop(columns=['country','ccode','location','region','sources','notes', 'protesteridentity','start_date','end_date', 
                              'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings'])

LogisticRegression_1 = LogisticRegression(solver='lbfgs', max_iter=1000)

# lr_arrests_1 = state_response_predictor(mass, features_1, 'arrests', lr_1)

In [102]:
# lr_arrests_1, f1_score_train_arrests, f1_score_test_arrests, precision_train, precision_test, recall_train, recall_test = state_response_predictor(mass, 
#                                                                                                                                    features_1, 
#                                                                                                                                    'arrests', 
#                                                                                                                                    LogisticRegression_1)

In [103]:
(sorted(list(mass.columns)))

['accomodation',
 'arrests',
 'arrests_predicted',
 'arrests_probability',
 'beatings',
 'ccode',
 'country',
 'country_Afghanistan',
 'country_Albania',
 'country_Algeria',
 'country_Angola',
 'country_Argentina',
 'country_Armenia',
 'country_Austria',
 'country_Azerbaijan',
 'country_Bahrain',
 'country_Bangladesh',
 'country_Belarus',
 'country_Belgium',
 'country_Benin',
 'country_Bolivia',
 'country_Bosnia',
 'country_Botswana',
 'country_Brazil',
 'country_Bulgaria',
 'country_Burkina Faso',
 'country_Burundi',
 'country_Cambodia',
 'country_Cameroon',
 'country_Canada',
 'country_Cape Verde',
 'country_Central African Republic',
 'country_Chad',
 'country_Chile',
 'country_China',
 'country_Colombia',
 'country_Comoros',
 'country_Congo Brazzaville',
 'country_Congo Kinshasa',
 'country_Costa Rica',
 'country_Croatia',
 'country_Cuba',
 'country_Cyprus',
 'country_Czech Republic',
 'country_Czechoslovakia',
 'country_Denmark',
 'country_Djibouti',
 'country_Dominican Republic',

In [104]:
# mass['arrests_predicted'], mass['arrests_probability'] = response_prediction_columns(lr_arrests_1, mass, features_1)

In [69]:
mass[['arrests', 'arrests_predicted', 'arrests_probability']][22:33]

arrests  arrests_predicted  arrests_probability
22        0                  0             0.379786
23        1                  1             0.606608
24        0                  0             0.209131
25        0                  1             0.594836
26        0                  0             0.162792
27        0                  0             0.392477
28        0                  0             0.120245
29        1                  1             0.536060
30        1                  0             0.472129
31        0                  0             0.211884
32        1                  1             0.584323

In [110]:
mass[['arrests', 'arrests_predicted', 'arrests_probability', 'accomodation', 'accomodation_predicted', 'accomodation_probability']][:100]

arrests  arrests_predicted  arrests_probability  accomodation  \
0         0                  0             0.104568             0   
1         0                  0             0.191000             0   
2         0                  0             0.205189             0   
3         0                  1             0.533197             1   
4         1                  0             0.421843             1   
..      ...                ...                  ...           ...   
95        0                  0             0.415251             0   
96        0                  0             0.413122             0   
97        0                  0             0.410995             0   
98        0                  0             0.408873             0   
99        0                  0             0.406753             0   

    accomodation_predicted  accomodation_probability  
0                        0                  0.174497  
1                        0                  0.117303  
2                        0                  0.052175  
3                        0                  0.156799  
4                        0                  0.085363  
..                     ...                       ...  
95                       0                  0.000388  
96                       0                  0.000387  
97                       0                  0.000385  
98                       0                  0.000383  
99                       0                  0.000382  

[100 rows x 6 columns]

In [113]:
mass[ (mass['accomodation'] == mass['accomodation_predicted']) & mass['accomodation']==1 ][['accomodation', 'accomodation_predicted', 'accomodation_probability']]

accomodation  accomodation_predicted  accomodation_probability
2112              1                       1                  0.516572
2186              1                       1                  0.549651
2474              1                       1                  0.532235
7070              1                       1                  0.586965
10393             1                       1                  0.577717
10631             1                       1                  0.601283
10633             1                       1                  0.627362
10635             1                       1                  0.593241
10661             1                       1                  0.506783
10664             1                       1                  0.501448
10667             1                       1                  0.526575
14781             1                       1                  0.567069
15179             1                       1                  0.532861
15195             1                       1                  0.518128
15198             1                       1                  0.548631

*Now that we've tested it on `arrests`, below I apply the the `state_response_predictor` function to the other six state response. This will create a column in the original `mass` DataFrame for the binary prediction and the probability for each response.*

Responses: `'arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings'`

In [119]:
mass['accomodation_predicted'], mass['accomodation_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features_1, 'accomodation', LogisticRegression_1)

accomodation- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.03843074459567654
accomodation- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.053571428571428575
accomodation- Precision for LogisticRegression(max_iter=1000) model, train set: 0.5454545454545454
accomodation- Precision for LogisticRegression(max_iter=1000) model, test set: 0.6
accomodation- Recall for LogisticRegression(max_iter=1000) model, train set: 0.01991701244813278
accomodation- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.028037383177570093
LogisticRegression(max_iter=1000)


In [114]:
mass['beatings_predicted'], mass['beatings_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features_1, 'beatings', LogisticRegression_1)

beatings- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.11881188118811882
beatings- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.14444444444444446
beatings- Precision for LogisticRegression(max_iter=1000) model, train set: 0.65625
beatings- Precision for LogisticRegression(max_iter=1000) model, test set: 0.65
beatings- Recall for LogisticRegression(max_iter=1000) model, train set: 0.06531881804043546
beatings- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.08125
LogisticRegression(max_iter=1000)


In [115]:
mass['crowddispersal_predicted'], mass['crowddispersal_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features_1, 'crowddispersal', LogisticRegression_1)

crowddispersal- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.657023000144655
crowddispersal- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.6085946573751453
crowddispersal- Precision for LogisticRegression(max_iter=1000) model, train set: 0.7373376623376623
crowddispersal- Precision for LogisticRegression(max_iter=1000) model, test set: 0.6649746192893401
crowddispersal- Recall for LogisticRegression(max_iter=1000) model, train set: 0.5924863031567963
crowddispersal- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.5610278372591007
LogisticRegression(max_iter=1000)


In [116]:
mass['ignore_predicted'], mass['ignore_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features_1, 'ignore', LogisticRegression_1)

ignore- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.794378288562725
ignore- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.7804335742078933
ignore- Precision for LogisticRegression(max_iter=1000) model, train set: 0.7336317135549872
ignore- Precision for LogisticRegression(max_iter=1000) model, test set: 0.7225939269171384
ignore- Recall for LogisticRegression(max_iter=1000) model, train set: 0.8660929951690821
ignore- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.8483383685800604
LogisticRegression(max_iter=1000)


In [117]:
mass['killings_predicted'], mass['killings_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features_1, 'killings', LogisticRegression_1)

killings- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.17120622568093383
killings- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.12435233160621761
killings- Precision for LogisticRegression(max_iter=1000) model, train set: 0.5789473684210527
killings- Precision for LogisticRegression(max_iter=1000) model, test set: 0.48
killings- Recall for LogisticRegression(max_iter=1000) model, train set: 0.1004566210045662
killings- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.07142857142857142
LogisticRegression(max_iter=1000)


In [118]:
mass['shootings_predicted'], mass['shootings_probability'], f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = \
state_response_predictor(mass, features_1, 'shootings', LogisticRegression_1)

shootings- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.21176470588235294
shootings- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.1722488038277512
shootings- Precision for LogisticRegression(max_iter=1000) model, train set: 0.5689655172413793
shootings- Precision for LogisticRegression(max_iter=1000) model, test set: 0.4864864864864865
shootings- Recall for LogisticRegression(max_iter=1000) model, train set: 0.13009198423127463
shootings- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.10465116279069768
LogisticRegression(max_iter=1000)


In [125]:
(list(mass.columns))

['id',
 'country',
 'ccode',
 'region',
 'protestnumber',
 'protesterviolence',
 'location',
 'protesteridentity',
 'sources',
 'notes',
 'partipants_number',
 'pop_male',
 'pop_female',
 'pop_total',
 'pop_density',
 'prosperity_2020',
 'country_Afghanistan',
 'country_Albania',
 'country_Algeria',
 'country_Angola',
 'country_Argentina',
 'country_Armenia',
 'country_Austria',
 'country_Azerbaijan',
 'country_Bahrain',
 'country_Bangladesh',
 'country_Belarus',
 'country_Belgium',
 'country_Benin',
 'country_Bolivia',
 'country_Bosnia',
 'country_Botswana',
 'country_Brazil',
 'country_Bulgaria',
 'country_Burkina Faso',
 'country_Burundi',
 'country_Cambodia',
 'country_Cameroon',
 'country_Canada',
 'country_Cape Verde',
 'country_Central African Republic',
 'country_Chad',
 'country_Chile',
 'country_China',
 'country_Colombia',
 'country_Comoros',
 'country_Congo Brazzaville',
 'country_Congo Kinshasa',
 'country_Costa Rica',
 'country_Croatia',
 'country_Cuba',
 'country_Cyprus'

In [122]:
pwd

'/Users/emilynaftalin/Data_Science/General Assembly/dsi/projects/Mass-Protests/users'

In [126]:
predictions_df = mass[['id','arrests', 'arrests_predicted', 'arrests_probability', 'accomodation', 'accomodation_predicted', 'accomodation_probability', 
      'beatings', 'beatings_predicted', 'beatings_probability', 'crowddispersal', 'crowddispersal_predicted', 'crowddispersal_probability', 
      'ignore', 'ignore_predicted', 'ignore_probability', 'killings', 'killings_predicted', 'killings_probability',
      'killings', 'killings_predicted', 'killings_probability']].copy()

In [127]:
predictions_df.head()

id  arrests  arrests_predicted  arrests_probability  accomodation  \
0  201990001        0                  0             0.104568             0   
1  201990002        0                  0             0.191000             0   
2  201990003        0                  0             0.205189             0   
3  201990004        0                  1             0.533197             1   
4  201990005        1                  0             0.421843             1   

   accomodation_predicted  accomodation_probability  beatings  \
0                       0                  0.174497         0   
1                       0                  0.117303         0   
2                       0                  0.052175         0   
3                       0                  0.156799         0   
4                       0                  0.085363         0   

   beatings_predicted  beatings_probability  ...  crowddispersal_probability  \
0                   0              0.000192  ...                    0.078412   
1                   0              0.000307  ...                    0.104708   
2                   0              0.000267  ...                    0.096429   
3                   0              0.000359  ...                    0.379723   
4                   0              0.000595  ...                    0.452339   

   ignore  ignore_predicted  ignore_probability  killings  killings_predicted  \
0       1                 1            0.755349         0                   0   
1       1                 1            0.692979         0                   0   
2       1                 1            0.687309         0                   0   
3       0                 0            0.055110         0                   0   
4       0                 0            0.138899         0                   0   

   killings_probability  killings  killings_predicted  killings_probability  
0              0.006260         0                   0              0.006260  
1              0.007246         0                   0              0.007246  
2              0.008844         0                   0              0.008844  
3              0.103104         0                   0              0.103104  
4              0.041307         0                   0              0.041307  

[5 rows x 22 columns]

In [128]:
predictions_df.to_csv('../data/04_predictions_only.csv', index=False)

In [86]:
model_performance_dict = {}

# ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']
# ['arrests_BC', 'accomodation_BC', 'beatings_BC', 'crowddispersal_BC', 'ignore_BC', 'killings_BC', 'shootings_BC']
 
targets = ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']    
    
for target in targets:
    
    predicted_column, probability_column, f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = state_response_predictor(
                                                                                                                                mass, 
                                                                                                                                features_1, 
                                                                                                                                classifier, 
                                                                                                                                LogisticRegression_1)
    
    model_performance_dict[target].append(target)
#     model_performance_dict['class_balance'].append(mass[classifier].value_counts(normalize=True))
    model_performance_dict['model'].append(model)
    model_performance_dict['f1_score_train'].append(f1_score_train)
    model_performance_dict['f1_score_test'].append(f1_score_test)
    model_performance_dict['precision_train'].append(precision_train)
    model_performance_dict['precision_test'].append(precision_test)
    model_performance_dict['recall_train'].append(recall_train)
    model_performance_dict['recall_test']
    
model_performance_df = pd.DataFrame(model_performance_dict)

arrests- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.2027871215761653
arrests- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.18148820326678766
arrests- Precision for LogisticRegression(max_iter=1000) model, train set: 0.5368956743002544
arrests- Precision for LogisticRegression(max_iter=1000) model, test set: 0.5434782608695652
arrests- Recall for LogisticRegression(max_iter=1000) model, train set: 0.125
arrests- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.10893246187363835
LogisticRegression(max_iter=1000)


KeyError: 'arrests'

In [90]:
model_performance_dict = {}
 
for classifier in ['arrests','accomodation','beatings','crowddispersal','ignore','killings','shootings']:
    
    predicted_column, probability_column, f1_score_train, f1_score_test, precision_train, precision_test, recall_train, recall_test = state_response_predictor(
                                                                                                                                mass, 
                                                                                                                                features_1, 
                                                                                                                                classifier, 
                                                                                                                                LogisticRegression_1)
     
    model_performance_dict['Classifier'] = classifier
#     model_performance_dict['class_balance'] = mass[classifier].value_counts(normalize=True))
    model_performance_dict['model'] = model
    model_performance_dict['f1_score_train'] = f1_score_train
    model_performance_dict['f1_score_test'] = f1_score_test
    model_performance_dict['precision_train'] = precision_train
    model_performance_dict['precision_test'] = precision_test
    model_performance_dict['recall_train'] = recall_train
    model_performance_dict['recall_test'] = recall_test
    
print(model_performance_dict)

arrests- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.2027871215761653
arrests- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.18148820326678766
arrests- Precision for LogisticRegression(max_iter=1000) model, train set: 0.5368956743002544
arrests- Precision for LogisticRegression(max_iter=1000) model, test set: 0.5434782608695652
arrests- Recall for LogisticRegression(max_iter=1000) model, train set: 0.125
arrests- Recall for LogisticRegression(max_iter=1000) model, teset set: 0.10893246187363835
LogisticRegression(max_iter=1000)
accomodation- F1_score for LogisticRegression(max_iter=1000) model, train set: 0.03522818254603683
accomodation- F1_score for LogisticRegression(max_iter=1000) model, test set: 0.041666666666666664
accomodation- Precision for LogisticRegression(max_iter=1000) model, train set: 0.5
accomodation- Precision for LogisticRegression(max_iter=1000) model, test set: 0.4666666666666667
accomodation- Recall for LogisticRegression

In [91]:
model_performance_df = pd.DataFrame(model_performance_dict)
model_performance_df

ValueError: If using all scalar values, you must pass an index

In [75]:
mass['arrests'].value_counts(normalize=True)[1]

0    0.858982
1    0.141018
Name: arrests, dtype: float64

In [76]:
mass['ignore'].value_counts(normalize=True)

1    0.543777
0    0.456223
Name: ignore, dtype: float64

In [77]:
mass['beatings'].value_counts(normalize=True)

0    0.947258
1    0.052742
Name: beatings, dtype: float64

In [78]:
mass['shootings'].value_counts(normalize=True)

0    0.938719
1    0.061281
Name: shootings, dtype: float64

In [79]:
mass['killings'].value_counts(normalize=True)

0    0.945813
1    0.054187
Name: killings, dtype: float64

In [80]:
mass['accomodation'].value_counts(normalize=True)

0    0.89977
1    0.10023
Name: accomodation, dtype: float64

In [81]:
mass['crowddispersal'].value_counts(normalize=True)

0    0.686897
1    0.313103
Name: crowddispersal, dtype: float64